In [19]:
import numpy as np 
import matplotlib.pyplot as plt
import pyrealsense2 as rs
import cv2
print("Environment Ready")

# Setup:
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)


# Create an object to read  
# from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera 
# is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 
    
# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 



# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
n_trial = 1
depthwriter = cv2.VideoWriter('depth_'+str(n_trial)+'.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         30, size) 


# Start streaming
pipe.start(cfg)


colorizer = rs.colorizer()
decimation = rs.decimation_filter(4)
threshold = rs.threshold_filter(0.15, 1.6)

# try:
# if len(start_test) != 0:
print("starting...")
try:
    while True:
        try:

            # Wait for a coherent pair of frames: depth and color
            frames = pipe.wait_for_frames()
            depth_frame = frames.get_depth_frame()

            
            # Decimation 
            depth_frame = decimation.process(depth_frame)

            # Thresholding
            depth_frame = threshold.process(depth_frame)
            
            
            colorized_depth = np.asanyarray(colorizer.colorize(depth_frame).get_data())
            
            key = cv2.waitKey(1)

            depthwriter.write(colorized_depth)

            cv2.imshow('Depth', colorized_depth)


            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                video.release() 
                depthwriter.release()
                break 
                
        except:
            pass
            break
finally:
    pipe.stop()
    cv2.destroyAllWindows()
    video.release() 
#     depthwriter.release()

Environment Ready
starting...


In [18]:
print(frame_width, frame_height)
print(colorized_depth.shape)
print(480/120)
print(640/212)

640 480
(120, 212, 3)
4.0
3.018867924528302


In [ ]:
import pyrealsense2 as rs
import cv2
import serial
import time
import csv
import numpy as np 



# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)



# Create an object to read  
# from camera 
video = cv2.VideoCapture(0) 
   
# We need to check if camera 
# is opened previously or not 
if (video.isOpened() == False):  
    print("Error reading video file") 
    
# We need to set resolutions. 
# so, convert them from float to integer. 
frame_width = int(video.get(3)) 
frame_height = int(video.get(4)) 
   
size = (frame_width, frame_height) 
   
# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
n_trial = 12
depthwriter = cv2.VideoWriter('fcss_leo_depth_test'+str(n_trial)+'.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         30, size) 


colorwriter = cv2.VideoWriter('fcss_leo_color_test'+str(n_trial)+'.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         30, size) 

# Start streaming
pipeline.start(config)

print("type anything to start")
start_test = input()


addr = "COM4" ## serial port to read data from
baud = 9600 ## baud rate for instrument

ser = serial.Serial(
    port = addr,\
    baudrate = baud)

fcss_filename = "fcss_leo_load_test"+str(n_trial)+".csv"
datafile=open(fcss_filename, 'a')


# try:
# if len(start_test) != 0:
print("starting...")
try:
    while True:
        try:
            ser_bytes = ser.readline()
            decoded_bytes = (ser_bytes[0:len(ser_bytes)-2].decode("utf-8").split(","))

            print(decoded_bytes)

            write_line = ",".join(decoded_bytes) + str("\n")
            datafile.write(write_line)

            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            color_frame = frames.get_color_frame()
            if not depth_frame or not color_frame:
                continue

            # Convert images to numpy arrays
            depth_image = np.asanyarray(depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

            key = cv2.waitKey(1)

            colorwriter.write(color_image)
            depthwriter.write(depth_colormap)

            cv2.imshow('Depth', depth_colormap)
            cv2.imshow('RGB', color_image)


            # Press esc or 'q' to close the image window
            if key & 0xFF == ord('q') or key == 27:
                cv2.destroyAllWindows()
                video.release() 
                colorwriter.release()
                depthwriter.release()
                break 
                
        except:
            pass
            break
finally:
    ser.close()
    datafile.close()
    pipeline.stop()
    cv2.destroyAllWindows()
    video.release() 
    colorwriter.release()
    depthwriter.release()